# Calculate-extracellular-fluxes-from-time-series-of-extracellular-metabolites

In [38]:
import numpy as np
import pandas as pd
import cobra

## Read metadata and time series of extracellular metabolite data

In [4]:
data_file = '../../data/round2/ABF_Aniger_BMCA_2_DATA.xlsx'

In [5]:
pd.ExcelFile(data_file).sheet_names

['notes',
 'Extracellular data summary',
 'Multiomics metadata',
 'Extracellular_metabolite',
 'global_protein',
 'targeted_protein',
 'intracellular_metabolite']

In [65]:
metadata = pd.read_excel(data_file, sheet_name='Multiomics metadata', index_col=0)
metadata.head()

,HPLD_ID,Strain,ICE_ID,Genotype,replicate,Glucose consumed (g/L),3HP Titer (g/L),DCW (g/L),Yield (g 3HP/g glucose),Rate (g 3HP/Lh)
sample ID,,,,,,,,,,
SF ABF180_1_R1,SF ABF180_73,ATCC11414 Parent,ABF_008340,wild-type,1,84.460574,0.000000,0.6897,0.000000,0.000000
SF ABF180_1_R2,SF ABF180_74,ATCC11414 Parent,ABF_008340,wild-type,2,84.366936,0.000000,0.7402,0.000000,0.000000
SF ABF180_1_R3,SF ABF180_75,ATCC11414 Parent,ABF_008340,wild-type,3,84.442214,0.000000,0.7080,0.000000,0.000000
SF ABF180_2_R1,SF ABF180_76,3HP-9,ABF_008343,(βAl-3HP)+,1,82.195887,13.819039,0.4236,0.168123,0.086369
SF ABF180_2_R2,SF ABF180_77,3HP-9,ABF_008343,(βAl-3HP)+,2,79.569229,13.554043,0.4140,0.170343,0.084713


In [24]:
extracellular_data = pd.read_excel(data_file, sheet_name='Extracellular_metabolite', index_col=[1,2])
extracellular_data.drop(columns='SampleName', inplace=True)
extracellular_data = extracellular_data.unstack(level=1).loc[metadata.index]
extracellular_data.head()

Glucose  (g/L)            3-Hydroxy-propionic Acid  (g/L)  \
Hour                      92         160                             92    
sample ID                                                                  
SF ABF180_1_R1      48.588632  16.270834                        0.000000   
SF ABF180_1_R2      47.764114  15.507360                        0.000000   
SF ABF180_1_R3      45.913887  16.290058                        0.000000   
SF ABF180_2_R1      48.426258  17.660951                        8.413751   
SF ABF180_2_R2      50.379407  21.003452                        8.610409   

                          Ethanol (g/L)           Erythritol (g/L)            \
Hour                  160           92        160              92        160   
sample ID                                                                      
SF ABF180_1_R1   0.000000      3.025401  1.312897         0.562383  0.581895   
SF ABF180_1_R2   0.000000      3.530351  1.958199         0.505038  0.579471   
SF ABF180_1_R3   0.000000      3.547300  1.150784         0.490549  0.550716   
SF ABF180_2_R1  13.707921      2.282904  1.814826         0.423544  0.503140   
SF ABF180_2_R2  13.933967      2.334804  1.635427         0.444181  0.462430   

               Citric Acid (g/L)            
Hour                         92        160  
sample ID                                   
SF ABF180_1_R1          0.323520  0.447325  
SF ABF180_1_R2          0.254056  0.350558  
SF ABF180_1_R3          0.260730  0.349104  
SF ABF180_2_R1          0.207337  0.200077  
SF ABF180_2_R2          0.162638  0.146766

## Read iJB1325 with non-native genes added for 3HP production

In [2]:
model = cobra.io.load_json_model('../../models/iJB1325_HP.json')

In [60]:
met_dict = {
    'Glucose  (g/L)': 'GLCe',
    '3-Hydroxy-propionic Acid  (g/L)': '3hpp_e',
    'Ethanol (g/L)': 'ETHe',
    'Erythritol (g/L)': 'EOLe',
    'Citric Acid (g/L)': 'CITe'
}

In [61]:
for k, v in met_dict.items():
    m = model.metabolites.get_by_id(v)
    print(k, m.name, m.formula, m.formula_weight)

Glucose  (g/L) alpha-D-Glucose (extracellular) C6H12O6 180.15588
3-Hydroxy-propionic Acid  (g/L) 3-Hydroxypropanoate C3H6O3 90.07794
Ethanol (g/L) Ethanol (extracellular) C2H6O 46.06844
Erythritol (g/L) Erythritol (extracellular) C4H10O4 122.1198
Citric Acid (g/L) Citrate (extracellular) C6H8O7 192.12351999999998


## Calculate extracellular fluxes

In [62]:
extracellular_flux = pd.DataFrame(index=extracellular_data.index, columns=extracellular_data.columns.levels[0])
for k, v in met_dict.items():
    df = extracellular_data[k]
    extracellular_flux[k] = (df.apply(lambda x: np.polyfit(df.columns,x,1)[0], axis=1).div(metadata['DCW (g/L)'])/
                             model.metabolites.get_by_id(v).formula_weight*1000)

In [64]:
for k, v in met_dict.items():
    extracellular_flux.columns = extracellular_flux.columns.str.replace(k, v + ' (mmol/gDW/hr)')
extracellular_flux.head()

,GLCe (mmol/gDW/hr),3hpp_e (mmol/gDW/hr),ETHe (mmol/gDW/hr),EOLe (mmol/gDW/hr),CITe (mmol/gDW/hr)
sample ID,,,,,
SF ABF180_1_R1,-3.824936,0.000000,-0.792609,0.003407,0.013740
SF ABF180_1_R2,-3.557249,0.000000,-0.678005,0.012109,0.009979
SF ABF180_1_R3,-3.415471,0.000000,-1.080524,0.010234,0.009554
SF ABF180_2_R1,-5.928543,2.040397,-0.352736,0.022628,-0.001312
SF ABF180_2_R2,-5.792077,2.099299,-0.539260,0.005308,-0.002935


In [71]:
line_rates = pd.DataFrame(index=metadata.index, columns=['Omics Sample ID','Strain (ICE)','Genotype'])
line_rates.index.name = 'Line Name'
line_rates['Omics Sample ID'] = metadata.index
line_rates['Strain (ICE)'] = metadata['ICE_ID']
line_rates['Genotype'] = metadata['Genotype']
line_rates = line_rates.join(extracellular_flux)
line_rates.head()

,Omics Sample ID,Strain (ICE),Genotype,GLCe (mmol/gDW/hr),3hpp_e (mmol/gDW/hr),ETHe (mmol/gDW/hr),EOLe (mmol/gDW/hr),CITe (mmol/gDW/hr)
Line Name,,,,,,,,
SF ABF180_1_R1,SF ABF180_1_R1,ABF_008340,wild-type,-3.824936,0.000000,-0.792609,0.003407,0.013740
SF ABF180_1_R2,SF ABF180_1_R2,ABF_008340,wild-type,-3.557249,0.000000,-0.678005,0.012109,0.009979
SF ABF180_1_R3,SF ABF180_1_R3,ABF_008340,wild-type,-3.415471,0.000000,-1.080524,0.010234,0.009554
SF ABF180_2_R1,SF ABF180_2_R1,ABF_008343,(βAl-3HP)+,-5.928543,2.040397,-0.352736,0.022628,-0.001312
SF ABF180_2_R2,SF ABF180_2_R2,ABF_008343,(βAl-3HP)+,-5.792077,2.099299,-0.539260,0.005308,-0.002935


In [72]:
line_rates.to_csv('../../data/round2/Line_rates.csv')